In [ ]:
# Library installs

!pip install polars
!pip install xgboost
!pip install scikit-learn
!pip install joblib
!pip install numpy



In [ ]:
import polars as pl
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
from pathlib import Path
from typing import Dict, Any
import numpy as np
import joblib
from sklearn.metrics import (
    confusion_matrix,
    precision_recall_fscore_support,
    f1_score,
)


Data Preparation

In [ ]:
FEATURE_COLUMNS = [
    "is_internal_source_ip",
    "tcp_flag", "udp_flag", "icmp_flag", "ftp_flag", "http_flag", "https_flag", "ssh_flag",
    "firewall_log", "application_log", "ids_log",
    "bytes_transferred_scaled",
    "curl_flag", "windows_browser_flag", "mac_browser_flag", "nmap_script_flag", "sqlmap_flag",
    "question_mark_count", "dotdot_count", "backslash_count", "admin_keyword_count", "passwd_keyword_count", "bin_bash_keyword_count", "root_keyword_count", "hydra_keyword_count"
]

TARGET_COLUMN = "threat_label_encoded"

# Adjusted path for notebook location (notebooks/ folder)
FILE_PATH = "../data/cybersecurity_threat_detection_logs.csv"

def load_data(path: str) -> pl.LazyFrame:
    """
    Load raw logs lazily using Polars.
    """
    return pl.scan_csv(
        path,
        infer_schema_length=10_000,
        ignore_errors=True
    )


def select_and_cast(df: pl.LazyFrame) -> pl.LazyFrame:
    """Select required columns and enforce correct data types."""
    return (
        df.select([
            pl.col("source_ip").cast(pl.Utf8),
            pl.col("protocol").cast(pl.Categorical),
            pl.col("threat_label").cast(pl.Categorical),
            pl.col("log_type").cast(pl.Categorical),
            pl.col("bytes_transferred").cast(pl.Int64),
            pl.col("user_agent").cast(pl.Utf8),
            pl.col("request_path").cast(pl.Utf8),
        ])
    )    

def split_data(df: pl.DataFrame, seed: int = 42):
    """
    Split dataset into train / validation / test 70 - 10 - 20 with stratification by threat_label.
    """
    groups = df.group_by("threat_label")
    train_parts = []
    val_parts = []
    test_parts = []
    for group_key, subgroup in groups:
        subgroup = subgroup.sample(n=subgroup.height, seed=seed, shuffle=True)
        n = subgroup.height
        train_end = int(0.7 * n)
        val_end = int(0.8 * n)
        train_parts.append(subgroup.slice(0, train_end))
        val_parts.append(subgroup.slice(train_end, val_end - train_end))
        test_parts.append(subgroup.slice(val_end, n - val_end))
    train = pl.concat(train_parts)
    val = pl.concat(val_parts)
    test = pl.concat(test_parts)
    return train, val, test

def add_ip_flags(df: pl.DataFrame) -> pl.DataFrame:
    """Add internal/external IP flags for source_ip."""
    return df.with_columns(
        pl.col("source_ip")
        .str.starts_with("192.168.")
        .cast(pl.Int8)
        .alias("is_internal_source_ip")
    )

def add_protocol_type_features(df: pl.DataFrame) -> pl.DataFrame:
    """Abstract protocol types into behavioral flags."""
    protocols = ["TCP", "UDP", "ICMP", "FTP", "HTTP", "HTTPS", "SSH"]
    return df.with_columns([
        (pl.col("protocol") == p).cast(pl.Int8).alias(f"{p.lower()}_flag")
        for p in protocols
    ])

def add_treatment_label_encoding(df: pl.DataFrame) -> pl.DataFrame:
    """Encode threat_label into numerical labels: benign=0, suspicious=1, malicious=2."""
    return df.with_columns(
        pl.when(pl.col("threat_label") == "benign").then(0)
        .when(pl.col("threat_label") == "suspicious").then(1)
        .otherwise(2)
        .cast(pl.Int8)
        .alias("threat_label_encoded")
    )

def add_log_type_features(df: pl.DataFrame) -> pl.DataFrame:
    """Abstract log types into behavioral flags."""
    return df.with_columns([
        (pl.col("log_type") == "firewall").cast(pl.Int8).alias("firewall_log"),
        (pl.col("log_type") == "application").cast(pl.Int8).alias("application_log"),
        (pl.col("log_type") == "ids").cast(pl.Int8).alias("ids_log"),
    ])

def add_bytes_transferred_scaling(df: pl.DataFrame, mean: float, std: float) -> pl.DataFrame:
    """Standardize bytes_transferred using z-score normalization."""
    return df.with_columns(
        ((pl.col("bytes_transferred") - mean) / std)
        .alias("bytes_transferred_scaled")
    )

def add_user_agent_features(df: pl.DataFrame) -> pl.DataFrame:
    """Abstract user agent strings into behavioral flags."""
    ua = pl.col("user_agent").str.to_lowercase()
    return df.with_columns([
        ua.str.contains("curl", literal=True).cast(pl.Int8).alias("curl_flag"),
        ua.str.contains("windows", literal=True).cast(pl.Int8).alias("windows_browser_flag"),
        ua.str.contains("macintosh", literal=True).cast(pl.Int8).alias("mac_browser_flag"),
        ua.str.contains("nmap", literal=True).cast(pl.Int8).alias("nmap_script_flag"),
        ua.str.contains("sqlmap", literal=True).cast(pl.Int8).alias("sqlmap_flag"),
    ])

def add_path_structure_features(df: pl.DataFrame) -> pl.DataFrame:
    """Extract structural features from request_path."""
    rp = pl.col("request_path").str.to_lowercase()
    return df.with_columns([
        rp.str.count_matches("?", literal=True).alias("question_mark_count"),
        rp.str.count_matches("..", literal=True).alias("dotdot_count"),
        rp.str.count_matches("\\\\", literal=True).alias("backslash_count"),
        rp.str.count_matches("admin", literal=True).alias("admin_keyword_count"),
        rp.str.count_matches("passwd", literal=True).alias("passwd_keyword_count"),
        rp.str.count_matches("bin/bash", literal=True).alias("bin_bash_keyword_count"),
        rp.str.count_matches("root", literal=True).alias("root_keyword_count"),
        rp.str.count_matches("hydra", literal=True).alias("hydra_keyword_count"),
    ])

def apply_features(df: pl.DataFrame, mean: float, std: float) -> pl.DataFrame:
    """Apply all feature engineering steps."""
    df = add_ip_flags(df)
    df = add_protocol_type_features(df)
    df = add_treatment_label_encoding(df)
    df = add_log_type_features(df)
    df = add_bytes_transferred_scaling(df, mean, std)
    df = add_user_agent_features(df)
    df = add_path_structure_features(df)
    return df.select(FEATURE_COLUMNS + [TARGET_COLUMN])

In [ ]:
# Run Data Preparation
print("Loading data...")
DF_LAZY = load_data(FILE_PATH)
DF_LAZY = select_and_cast(DF_LAZY)

# Validate schema
required_cols = [
    "source_ip", "protocol", "threat_label",
    "log_type", "bytes_transferred", "user_agent", "request_path"
]
schema_names = DF_LAZY.collect_schema().names()
for col in required_cols:
    assert col in schema_names, f"Missing required column: {col}"
print("Schema validation passed!")

# Collect and split
print("Collecting and splitting data...")
df = DF_LAZY.collect()
train, val, test = split_data(df)
print(f"Train: {train.height}, Val: {val.height}, Test: {test.height}")

# Compute scaling parameters from training set
mean = train.select(pl.col("bytes_transferred").mean()).item()
std = train.select(pl.col("bytes_transferred").std()).item()
print(f"Bytes transferred - Mean: {mean:.2f}, Std: {std:.2f}")

# Apply features
print("Applying feature engineering...")
train = apply_features(train, mean, std)
val = apply_features(val, mean, std)
test = apply_features(test, mean, std)

print("Data preparation complete!")
print(f"Features: {FEATURE_COLUMNS}")
print(f"Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}")

Model Implementation

In [ ]:
def build_logistic_regression(config: Dict[str, Any]):
    """
    Logistic Regression using SGDClassifier with 'modified_huber' loss.
    """
    return SGDClassifier(
        loss="modified_huber", 
        penalty=config.get("penalty", "l2"),
        alpha=0.0001, 
        max_iter=2000,
        random_state=config.get("random_state", 42),
        class_weight=config.get("class_weight", None),
        n_jobs=13 
    )

def build_random_forest(config: Dict[str, Any]):
    """
    Docstring for build_random_forest
    
    :param config: Description
    :type config: Dict[str, Any]
    """
    return RandomForestClassifier(
        n_estimators=config.get("n_estimators", 300),
        max_depth=None,
        class_weight=config.get("class_weight", None),
        random_state=config.get("random_state", 42),
        n_jobs=13 
    )

def build_gradient_boosting(config: Dict[str, Any]):
    """
    XGBoost 2.0+ GPU configuration.
    Uses tree_method='hist' with device='cuda'.
    """
    is_gpu = config.get("device") == "gpu"
    
    return xgb.XGBClassifier(
        n_estimators=config.get("n_estimators", 400),
        learning_rate=0.05,
        max_depth=6,
        tree_method="hist",
        device="cuda" if is_gpu else "cpu",
        n_jobs=13,
        random_state=config.get("random_state", 42)
    )

def get_model(model_name: str, config: Dict[str, Any]):
    """
    Docstring for get_model
    
    :param model_name: Description
    :type model_name: str
    :param config: Description
    :type config: Dict[str, Any]
    """
    if model_name == "logistic_regression":
        return build_logistic_regression(config)
    elif model_name == "random_forest":
        return build_random_forest(config)
    elif model_name == "gradient_boosting":
        return build_gradient_boosting(config)
    else:
        raise ValueError(f"Unsupported model_name: {model_name}")

Train the Model


In [ ]:
# Adjusted paths for notebook location
ARTIFACT_DIR = Path("../artifacts")
MODEL_OUTPUT_DIR = Path("../models")
MODEL_OUTPUT_DIR.mkdir(exist_ok=True)
RANDOM_STATE = 42

def compute_class_weights(y: np.ndarray) -> Dict[int, float]:
    """
    Compute class weights inversely proportional to class frequency.
    """
    unique, counts = np.unique(y, return_counts=True)
    total = counts.sum()
    class_weight = {
        int(cls): float(total / (len(unique) * cnt))
        for cls, cnt in zip(unique, counts)
    }
    return class_weight

def train_model(model_name, model_config, X_train, y_train):
    """
    Train a model with the given configuration.
    """
    model = get_model(model_name, model_config)

    with joblib.parallel_backend('loky', n_jobs=10):
        if model_name == "gradient_boosting":
            cw_dict = model_config.get("class_weight")
            if cw_dict:
                row_weights = np.array([cw_dict[y] for y in y_train])
                model.fit(X_train, y_train, sample_weight=row_weights)
            else:
                model.fit(X_train, y_train)
        else:
            model.fit(X_train, y_train)
            
    return model

Evaluation 

In [ ]:

def evaluate_classification(
    model,
    X_val: np.ndarray,
    y_val: np.ndarray,
    model_name: str,
) -> Dict[str, Any]:
    """
    Evaluate a trained model using security-relevant metrics.

    Metrics Emphasized
    ------------------
    - Recall (malicious)  ← highest priority
    - Recall (suspicious)
    - Macro F1
    - Confusion matrix

    Parameters
    ----------
    model : trained model
        Must implement predict() and optionally predict_proba()
    X_val : np.ndarray
        Validation or test features
    y_val : np.ndarray
        True labels
    model_name : str
        Identifier for logging and reporting

    Returns
    -------
    metrics : dict
        Dictionary containing detailed evaluation metrics
    """

    # Predictions
    y_pred = model.predict(X_val)

    # Core metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        y_val,
        y_pred,
        labels=[0, 1, 2],  # benign, suspicious, malicious
        zero_division=0,
    )

    macro_f1 = f1_score(y_val, y_pred, average="macro")
    weighted_f1 = f1_score(y_val, y_pred, average="weighted")

    malicious_recall = recall[2]
    suspicious_recall = recall[1]

    cm = confusion_matrix(y_val, y_pred, labels=[0, 1, 2])

    # Structured report
    metrics = {
        "model_name": model_name,
        "macro_f1": macro_f1,
        "weighted_f1": weighted_f1,
        "malicious_recall": malicious_recall,
        "suspicious_recall": suspicious_recall,
        "per_class": {
            "benign": {
                "precision": precision[0],
                "recall": recall[0],
                "f1": f1[0],
                "support": support[0],
            },
            "suspicious": {
                "precision": precision[1],
                "recall": recall[1],
                "f1": f1[1],
                "support": support[1],
            },
            "malicious": {
                "precision": precision[2],
                "recall": recall[2],
                "f1": f1[2],
                "support": support[2],
            },
        },
        "confusion_matrix": cm,
    }

    print_evaluation(metrics)
    return metrics


# Reporting Utilities

def print_evaluation(metrics: Dict[str, Any]):
    """
    Print a concise, security-focused evaluation summary.
    """
    print("\n================ Evaluation =================")
    print(f"Model: {metrics['model_name']}")
    print(f"Macro F1          : {metrics['macro_f1']:.4f}")
    print(f"Weighted F1       : {metrics['weighted_f1']:.4f}")
    print(f"Malicious Recall  : {metrics['malicious_recall']:.4f}")
    print(f"Suspicious Recall : {metrics['suspicious_recall']:.4f}")
    print("============================================")


# Model Comparison & Selection

def summarize_results(results: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    """
    Rank models and select the best one based on security priorities.

    Ranking Strategy (in order)
    ---------------------------
    1. Highest malicious recall
    2. Highest suspicious recall
    3. Highest macro F1

    Parameters
    ----------
    results : dict
        Output from training loop in train.py

    Returns
    -------
    summary : dict
        Ranked models and selected best model
    """

    rankings = []

    for model_name, payload in results.items():
        metrics = payload["metrics"]
        rankings.append({
            "model": model_name,
            "malicious_recall": metrics["malicious_recall"],
            "suspicious_recall": metrics["suspicious_recall"],
            "macro_f1": metrics["macro_f1"],
        })

    # Sort by security priority
    rankings.sort(
        key=lambda x: (
            x["malicious_recall"],
            x["suspicious_recall"],
            x["macro_f1"],
        ),
        reverse=True,
    )

    best_model = rankings[0]["model"]

    print("\n=========== Model Ranking ===========")
    for rank, entry in enumerate(rankings, start=1):
        print(
            f"{rank}. {entry['model']} | "
            f"MalRec={entry['malicious_recall']:.4f} | "
            f"SusRec={entry['suspicious_recall']:.4f} | "
            f"MacroF1={entry['macro_f1']:.4f}"
        )
    print("====================================")

    return {
        "rankings": rankings,
        "best_model": best_model,
    }


## Run Training Pipeline

In [ ]:
# Prepare data for training
X_train = train.select(FEATURE_COLUMNS).to_numpy()
y_train = train.select(TARGET_COLUMN).to_numpy().ravel()
X_val = val.select(FEATURE_COLUMNS).to_numpy()
y_val = val.select(TARGET_COLUMN).to_numpy().ravel()
X_test = test.select(FEATURE_COLUMNS).to_numpy()
y_test = test.select(TARGET_COLUMN).to_numpy().ravel()

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Compute class weights
class_weight = compute_class_weights(y_train)
print(f"Class weights: {class_weight}")

# Define experiments
experiments = {
    "logistic_regression": {
        "class_weight": class_weight,
        "random_state": RANDOM_STATE,
    },
    "random_forest": {
        "n_estimators": 200,
        "class_weight": class_weight,
        "random_state": RANDOM_STATE,
    },
    "gradient_boosting": {
        "n_estimators": 400,
        "class_weight": class_weight,
        "device": "gpu",  # Change to "cpu" if no GPU available
        "random_state": RANDOM_STATE,
    },
}

# Train and evaluate all models
results = {}
for model_name, config in experiments.items():
    print(f"\n{'='*50}")
    print(f"Training model: {model_name}")
    print(f"{'='*50}")
    model = train_model(model_name, config, X_train, y_train)
    metrics = evaluate_classification(model, X_val, y_val, model_name)
    results[model_name] = {"model": model, "metrics": metrics}

# Rank and select best model
summary = summarize_results(results)
best_model_name = summary["best_model"]
best_model = results[best_model_name]["model"]

print(f"\n{'='*50}")
print(f"Selected best model: {best_model_name}")
print(f"{'='*50}")

# Final evaluation on test set
print("\nFinal Evaluation on Test Set:")
final_metrics = evaluate_classification(best_model, X_test, y_test, f"{best_model_name}_test")

# Save best model and metrics
with open(MODEL_OUTPUT_DIR / "best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)
with open(MODEL_OUTPUT_DIR / "final_metrics.pkl", "wb") as f:
    pickle.dump(final_metrics, f)

print(f"\nModel saved to {MODEL_OUTPUT_DIR / 'best_model.pkl'}")
print(f"Metrics saved to {MODEL_OUTPUT_DIR / 'final_metrics.pkl'}")